In [1]:
#Imports
from load_hyperparams import load_hyperparams
from predict_satellites import get_satellite_properties
from predict_orphans import get_orphan_satellite_properties
from fix_lmc_coords import get_lmc_coords, rotate_about_LMC
from astropy.coordinates import SkyCoord
from masks import load_mask, evaluate_mask

In [2]:
#Set parameters
params = {}

params['alpha'] = -1.4
params['sigma_M'] = 0.1
params['M50'] = 8.
params['sigma_mpeak'] = 0.35
params['B'] = 1.
params['A'] = 34
params['sigma_r'] = 0.3
params['n'] = 1.3

In [3]:
#Load hyperparameters
hparams, cosmo_params, orphan_params, halo_data, vpeak_Mr_interp = load_hyperparams()

In [4]:
#Return satellite properties for a particular MW-like host halo
satellite_properties = get_satellite_properties(halo_data[14], params, hparams, cosmo_params, vpeak_Mr_interp)
print(satellite_properties)

{'Mr': array([-1.81089041e+01, -1.68517549e+01, -1.21263877e+01, -1.19380950e+01,
       -1.42635599e+01, -1.37001657e+01, -9.53933293e+00, -9.13885022e+00,
       -9.77945834e+00, -9.76774084e+00, -6.15518565e+00, -8.67036143e+00,
       -6.86772523e+00, -7.91950960e+00, -1.22500265e+01, -9.47415735e+00,
       -6.50774892e+00, -8.86061236e+00, -6.56437679e+00, -9.06975382e+00,
       -7.61003717e+00, -1.06274299e+01, -5.86331232e+00, -6.19876486e+00,
       -9.20465819e+00, -5.09232816e+00, -6.00799436e+00, -3.24265187e+00,
       -4.74102394e+00, -6.72883282e+00, -4.81638262e+00, -7.00400874e+00,
       -4.07468273e+00, -3.30046187e+00, -6.45398744e+00, -4.38791553e+00,
       -9.59427728e+00, -9.83383093e+00, -9.94647754e+00, -3.67799159e+00,
       -4.14312716e+00, -3.14602160e+00, -6.56944943e+00, -3.47137912e+00,
       -6.61196584e+00, -6.02725901e+00, -6.53852746e+00, -3.12810823e+00,
       -7.67807226e+00, -3.40002664e+00, -1.87759454e+00, -5.05348329e+00,
       -5.19807366

In [5]:
#Return orphan satellite properties for a particular host halo
orphan_satellite_properties = get_orphan_satellite_properties(halo_data[14], params, hparams, 
                                                              cosmo_params, vpeak_Mr_interp)
print(orphan_satellite_properties)

{'Mr': array([-2.72916818e+00, -5.53536673e+00, -5.63918709e+00, -6.38959385e+00,
       -8.61378604e-01,  2.38922215e+00, -1.16882837e+00, -3.80924405e-01,
        2.57831898e+00,  1.89136429e+00, -8.73659771e-02,  2.42792155e+00,
        2.65814284e+00,  2.81953172e+00,  1.79580574e+00,  2.61210466e+00,
        2.21682144e+00,  2.85926444e+00, -2.93046974e-02,  2.65483310e+00,
        2.60291206e+00,  2.41830856e+00, -7.24020696e-01,  2.21269526e+00,
        2.60511825e+00,  2.80255745e+00,  1.64747024e+00,  2.83117774e+00,
        2.28159177e+00,  2.43437175e+00,  2.55285857e+00,  2.12780114e+00,
        2.83533661e+00,  3.02596953e+00, -5.28640652e-02,  2.67316164e+00,
        1.19775795e+00,  2.29221164e+00,  2.38071967e+00,  2.82898726e+00,
        2.74066708e+00, -2.73505493e+00,  2.24135943e+00,  1.34903850e+00,
        6.69444999e-01,  2.74377964e+00,  2.54753247e+00, -1.00118780e+00,
        3.11796969e+00, -5.66246048e-01,  2.52516114e+00,  2.52512724e+00,
        2.70974915

In [6]:
#Rotate about LMC
lmc_ind = 0
lmc_cartesian_coords = get_lmc_coords(halo_data[14],cosmo_params,lmc_ind)
satellite_properties_rotated = rotate_about_LMC(satellite_properties,halo_data[0],cosmo_params,
                                                lmc_cartesian_coords,lmc_ind,observer_ind=0)

In [7]:
#Precompute satellite properties and flags for each observer location
satellite_properties_list = []
DES_flags_list = []
PS1_flags_list = []
Halo_ra_list = []
Halo_dec_list = []

print('loading masks ...')
des_mask = load_mask('des')
ps1_mask = load_mask('ps1')
print('done')

for i in range(6):
    #Get satellite properties
    satellite_properties = get_satellite_properties(halo_data[14], params, hparams, cosmo_params, vpeak_Mr_interp)
    satellite_properties_rotated = rotate_about_LMC(satellite_properties,halo_data[14],cosmo_params,
                                                lmc_cartesian_coords,lmc_ind,i)
    #Transform to sky coordinates
    Halo_sky_coord = SkyCoord(x=satellite_properties_rotated['rotated_pos'][:,0], 
                      y=satellite_properties_rotated['rotated_pos'][:,1], 
                      z=satellite_properties_rotated['rotated_pos'][:,2], 
                      unit='kpc', representation_type='cartesian').spherical
    satellite_properties_rotated['ra'] = Halo_sky_coord.lon.degree
    satellite_properties_rotated['dec'] = Halo_sky_coord.lat.degree
    #Assign flags 
    satellite_properties_rotated['DES_flags'] = evaluate_mask(Halo_sky_coord.lon.degree,Halo_sky_coord.lat.degree,
                                                               des_mask)
    satellite_properties_rotated['PS1_flags'] = evaluate_mask(Halo_sky_coord.lon.degree,Halo_sky_coord.lat.degree,
                                                               ps1_mask)
    #Append
    satellite_properties_list.append(satellite_properties_rotated)

loading masks ...
NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
done
